In [1]:
from xgboost import XGBClassifier
import warnings
from tabpfn import TabPFNClassifier
import numpy as np
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from data_prep_utils import *
from evaluate import *
from load_models import *
import matplotlib.pyplot as plt
import torch
import openml

In [2]:
path = "datasets/data_all.csv"
data, labels = get_microbiome(path)
data = top_non_zero(data)
data, labels = unison_shuffled_copies(data, labels)

In [3]:
cv = 3
n_optim = 10
ft_epochs = 10
sampling = None
metrics = metrics = ["accuracy", "precision", "recall", "roc_auc"]
models = [
    XGBClassifier(n_estimators=5, max_depth=5, learning_rate=1, objective='binary:logistic'),
    XGBoostOptim(n_optim=n_optim),
    LogisticRegression(max_iter=500), 
    TabPFNClassifier(device='cpu', N_ensemble_configurations=3),
    TabForestPFNClassifier("saved_models/tabforest/mix600k/tabforestpfn.pt", "saved_models/tabforest/mix600k/config_run.yaml")
]
results = pd.DataFrame(np.zeros((len(models), len(metrics))), 
                       index=[m.__class__.__name__ for m in models],
                      columns=metrics)

for ii, model in enumerate(models):
    results.iloc[ii,:] = cross_validate_sample(model, data, labels, metrics, cv, sampling)

C:\Users\lenna\anaconda3\envs\master3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\lenna\anaconda3\envs\master3\Lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
C:\Users\lenna\anaconda3\envs\master3\Lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we w

KeyboardInterrupt: 

In [ ]:
print(results)

In [ ]:
data, labels = oversample(data, labels)